In [1]:
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
import time
import re
from pprint import pprint
import glob
import os
import warnings
warnings.filterwarnings('ignore')


# Read Player Database

In [2]:
players_df = pd.read_csv('Player Database - Singapore.csv',
                        names=('ID','First Name','Last Name'),usecols=[0,1,2])

In [7]:
csv_dir = "C:\\misc\\scba-techcomm\\masterpoints\\csv_2019\\"

# Helper Functions

## Player Number Lookup

In [3]:
def find_player_numbers(name, table_df):
    name_words = name.split(' ')
    last_name = name_words[-1]
    first_name = ' '.join(name_words[:-1])
    
    try:
        player_id = int(players_df[(players_df['First Name']==first_name) & (players_df['Last Name']==last_name)]['ID'].item())
        return player_id
    except:
        return None

In [17]:
def ext_meta(df_meta):
    event_meta_df = pd.DataFrame()
    event_meta_df['Date'] = df_meta.loc['#Date']
    event_meta_df['Event'] = df_meta.loc['#Title']
    event_meta_df['Event_ID'] = df_meta.loc['Event Number']
    event_meta_df['Director'] = df_meta.loc['#Director']
    event_meta_df['Boards'] = int(df_meta.loc['#Boards'].value)
    try:
        event_meta_df['Tables'] = int(df_meta.loc['#Tables'].value)
    except:
        event_meta_df['Tables'] = 0
    try:
        event_meta_df['Rounds'] = int(df_meta.loc['#Rounds'].value)
    except:
        event_meta_df['Rounds'] = 0
    try:
        event_meta_df['Matches'] = int(df_meta.loc['#Matches'].value)
    except:
        event_meta_df['Matches'] = 0
    try:
        event_meta_df['Phantom'] = int(df_meta.loc['#Phantom'].value)
    except:
        event_meta_df['Phantom'] = 0
    
    event_meta_df.reset_index()
    
    return event_meta_df

# Read all metadata

In [35]:
all_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"*.csv"):
    print(csv_file)
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20,)
    event_meta_df = ext_meta(df_meta)
    
    filename = csv_file.split('\\')[-1]
    event_meta_df['Filename'] = filename
    event_meta_df['Source'] = filename[0:2]
    
    all_event_meta_df = all_event_meta_df.append(event_meta_df,ignore_index=True)

all_event_meta_df['Date'] = pd.to_datetime(all_event_meta_df['Date'],dayfirst=True)

C:\misc\scba-techcomm\masterpoints\csv_2019\ISNov23-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr01-2019-244.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr04-2019-245.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr05-2019-246.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr08-2019-247.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr11-2019-248.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr12-2019-249.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr15-2019-250.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr18-2019-1.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-14.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-5.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr22-2019-6.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr25-2019-7.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr26-2019-8.csv
C:\misc\scba-techcomm\masterpoints\csv_2

C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov11-2019-74.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov14-2019-76.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov15-2019-77.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov18-2019-78.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov21-2019-79.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov22-2019-80.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov25-2019-81.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov28-2019-82.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSNov29-2019-83.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct03-2019-69.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct04-2019-70.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct05-2019-75.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct07-2019-87.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct10-2019-88.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSOct11-2019-89.csv
C:\misc\scba-techcomm\masterpoints\csv_2

In [36]:
all_event_meta_df

,Date,Event,Event_ID,Director,Boards,Tables,Rounds,Matches,Phantom,Filename,Source
0,2019-11-23,SCBA Individual Game 2019,2,Shen Ting Ang,24,2,8,0,0,ISNov23-2019-2.csv,IS
1,2019-04-01,Monday Pairs,244,Greta Chai,24,11,12,0,0,PSApr01-2019-244.csv,PS
2,2019-04-04,Thursday Pairs,245,Greta Chai,24,7,12,0,0,PSApr04-2019-245.csv,PS
3,2019-04-05,Friday Pairs,246,Peter Haw,24,9,12,0,0,PSApr05-2019-246.csv,PS
4,2019-04-08,Monday Pairs,247,Tan Wen Yuan,24,8,12,0,0,PSApr08-2019-247.csv,PS
5,2019-04-11,Thursday Pairs,248,Greta Chai,24,10,12,0,5,PSApr11-2019-248.csv,PS
6,2019-04-12,Friday Pairs,249,Tan Wen Yuan,24,12,12,0,0,PSApr12-2019-249.csv,PS
7,2019-04-15,Monday Pairs,250,Greta Chai,24,7,12,0,12,PSApr15-2019-250.csv,PS
8,2019-04-18,Thursday Pairs,1,Greta Chai,24,9,12,0,0,PSApr18-2019-1.csv,PS
9,2019-04-19,Easter Congress IMP Pairs,14,Wei Kang Gan,28,12,14,0,24,PSApr19-2019-14.csv,PS


In [37]:
all_event_meta_df.sort_values('Date',inplace=True)

In [38]:
all_event_meta_df.to_csv('all_meta.csv', index=False)

In [ ]:
def calc_match_mp(boards, in_df):
    
    

In [5]:
def calc_tables_phantom(df_meta):
    # Calculate Number of tables and phantom pair exists
    n_tables = int(df_meta.loc['#Tables'].value)
    n_boards = int(df_meta.loc['#Boards'].value)
    try:
        n_phantom = int(df_meta.loc['#Phantom'].value)
    except:
        n_phantom = 0
    if n_phantom > 0:
        n_phantom = 1
    return n_tables, n_boards, n_phantom

In [ ]:
def calc_rank_mp(in_df, boards, tables, scale='A', qualifinal=False, single_winner=True):
    """
    calc_mp: calculates masterpoints for ranks
    Inputs -
    in_df: DataFrame of results (which does not have Masterpoints column)
    boards: Number of boards played
    tables: Number of FULL tables
    scale: 'Club', 'District', 'County', 'Regional', 'National' (default = 'Club')
    single_winner: True if 1-winner pairs, False if 2-winner pairs (default=True)
    Returns -
    out_df: DataFrame of only masterpoint winners with 'MPTS' column appended
    
    Reference: Masterpoint Handbook Scale A-E
    """
    if scale=='E':
        scale_factor = 4
    elif scale =='D':
        scale_factor = 3
    elif scale =='C':
        scale_factor = 2
    elif scale =='B':
        scale_factor = 1.5
    elif scale == 'A':
        scale_factor = 1
    else:
        scale_factor = 1
    
    
    if single_winner:
        base_factor = 6*scale_factor
        if boards < 12:
            n_awards = 0
            return None
        elif boards >= 12 and boards < 18:
            factor = 2
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        elif boards >= 18 and boards < 36:
            factor = 1.5
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        else:
            n_awards = tables # Factor is 1
        
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        return out_df
        
    else:
        base_factor = 10*scale_factor
        if boards < 12:
            return None
        elif boards >= 12 and boards < 18:
            factor = 4
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        elif boards >= 18 and boards < 36:
            factor = 3
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        else:
            factor = 2
            if tables >=2:
                n_awards = round(tables/factor, 0)
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        return out_df

In [6]:
def calc_mp(in_df, boards, tables, scale='Club', single_winner=True):
    """
    calc_mp: calculates masterpoints
    Inputs -
    in_df: DataFrame of results (which does not have Masterpoints column)
    boards: Number of boards played
    tables: Number of FULL tables
    scale: 'Club', 'District', 'County', 'Regional', 'National' (default = 'Club')
    single_winner: True if 1-winner pairs, False if 2-winner pairs (default=True)
    Returns -
    out_df: DataFrame of only masterpoint winners with 'MPTS' column appended
    
    Reference: Masterpoint Handbook Scale A-E
    """
    if scale=='National':
        scale_factor = 4
    elif scale =='Regional':
        scale_factor = 3
    elif scale =='County':
        scale_factor = 2
    elif scale =='District':
        scale_factor = 1.5
    elif scale == 'Club':
        scale_factor = 1
    else:
        scale_factor = 1
    
    
    if single_winner:
        base_factor = 6*scale_factor
        if boards < 12:
            n_awards = 0
            return None
        elif boards >= 12 and boards < 18:
            factor = 2
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        elif boards >= 18 and boards < 36:
            factor = 1.5
            if tables >=3:
                n_awards = math.ceil(tables/factor)
            else:
                n_awards = 0
                return None
        else:
            n_awards = tables # Factor is 1
        
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        return out_df
        
    else:
        base_factor = 10*scale_factor
        if boards < 12:
            return None
        elif boards >= 12 and boards < 18:
            factor = 4
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        elif boards >= 18 and boards < 36:
            factor = 3
            # Limit at 30 tables
            if tables > 30:
                tables = 30
            if tables >=3:
                n_awards = round(tables/factor, 0)
            else:
                return None
        else:
            factor = 2
            if tables >=2:
                n_awards = round(tables/factor, 0)
        out_df = in_df[in_df['Position']<=n_awards]
        out_df['MPTS'] = base_factor*(n_awards+1-out_df['Position'])
        return out_df

In [8]:
def ext_winning_names(df, boards, tables, scale='Club', single_winner=True):
    if 'MPTS' in df.columns:
        df_winners = df[df['MPTS'].astype(int)>0]
    else:
        df_winners = calc_mp(df, boards, tables, scale, single_winner)
    
    if df_winners is not None:
        df_winners1 = df_winners.drop('Name2', axis=1)
        df_winners2 = df_winners.drop('Name1', axis=1)
    
        df_winners1.rename(columns={'Name1': 'Name'},inplace=True)
        df_winners2.rename(columns={'Name2': 'Name'},inplace=True)
    
        out_df = df_winners1.append(df_winners2)
        return out_df
    else:
        return None

In [9]:
def find_player_numbers(name, table_df):
    name_words = name.split(' ')
    last_name = name_words[-1]
    first_name = ' '.join(name_words[:-1])
    
    try:
        player_id = int(players_df[(players_df['First Name']==first_name) & (players_df['Last Name']==last_name)]['ID'].item())
        return player_id
    except:
        return None
    

In [11]:
def read_from_csv(csv_file):
    # Read Metadata first - take first 20 rows
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20)
    
    n_tables, n_boards, n_phantom = calc_tables_phantom(df_meta)
    
    if int(df_meta.loc['#Winners'].value)!=1:
        single_winner=False
    else:
        single_winner=True
    
    max_pair = 2 * n_tables - n_phantom
    
    # Read actual results
    df = pd.read_csv(csv_file, sep=';',comment='#',header=1, nrows=50, error_bad_lines=False,warn_bad_lines=False)
    if 'Position' in df['Position'].values:
        df.drop(df[df['Position']=='Position'].index, inplace=True) # Clear up extra header for 2-winner
    pairs_df = df.iloc[0:int(max_pair)].sort_values('Position',ascending=True)
    
    if 'NS' in pairs_df['Position'].values:
        pairs_df.drop(pairs_df[pairs_df['Position']=='NS'].index, inplace=True) # Clear up extra header for missing phantom
        n_tables = n_tables - 1
        
    pairs_df['Position'] = pairs_df['Position'].astype('int') # Convert position to int
    
    out_df = ext_winning_names(pairs_df, n_boards, n_tables, 'Club', single_winner)
    
    if out_df is not None:
    
        out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
    
        out_df['Date'] = df_meta.loc['#Date'].value
        out_df['Event'] = df_meta.loc['#Title'].value
        out_df['Event_ID'] = df_meta.loc['Event Number'].value
    
        event_meta_df = ext_meta(df_meta)
    
        return out_df, event_meta_df
    else:
        event_meta_df = ext_meta(df_meta)
        return None, event_meta_df

In [258]:
pairs_mp_df = pd.DataFrame()
pairs_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"PS*.csv"):
    print(csv_file)
    out_df, event_meta_df = read_from_csv(csv_file)
    pairs_mp_df = pairs_mp_df.append(out_df,ignore_index=True)
    pairs_event_meta_df = pairs_event_meta_df.append(event_meta_df,ignore_index=True)

C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr01-2019-244.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr04-2019-245.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr05-2019-246.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr08-2019-247.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr11-2019-248.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr12-2019-249.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr15-2019-250.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr18-2019-1.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-14.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-2.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr19-2019-5.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr22-2019-6.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr25-2019-7.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr26-2019-8.csv
C:\misc\scba-techcomm\masterpoints\csv_2019\PSApr29-2019-9.csv
C:\misc\scba-techcomm\masterpoints\csv_2

# Fix Scaling

In [208]:
special_event_meta_df = pairs_event_meta_df[(pairs_event_meta_df['Event']!='Monday Pairs')
                   & (pairs_event_meta_df['Event']!='Thursday Pairs')
                   & (pairs_event_meta_df['Event']!='Friday Pairs')]

In [210]:
special_event_meta_df.to_csv('PS_special_meta.csv',index=False)

In [257]:
special_event_meta_df = pd.read_csv('PS_special_meta.csv')

In [224]:
special_event_meta_df

,Date,Event,Event_ID,Director,Boards,Tables,Rounds,Phantom,Scale
0,19/04/2019,Easter Congress IMP Pairs,14,Wei Kang Gan,28,12,14,24,National
1,19/04/2019,EC Mixed,2,Luwen Koh,28,10,14,0,National
2,19/04/2019,Open pairs,5,Luwen Koh,57,10,0,0,National
3,02/08/2019,Pesta Sukan Open Pairs,52,Raj Jayaram,26,8,13,0,National
4,16/12/2019,Christmas Pairs,110,Shen Ting Ang,22,16,0,0,District
5,11/02/2019,Hongbao Pairs,217,Siew Kheng Hun,36,18,10,0,District
6,23/02/2019,NTU Open Pairs,1,Luwen Koh,24,11,12,0,District
7,01/01/2019,Tuesday Pairs New Year Day,200,Greta Chai,24,7,12,12,District
8,12/01/2019,NUS Open Pairs,1,,32,16,14,0,District
9,13/07/2019,Singapore Mixed Pairs,43,Yang Gan,54,9,0,0,National


In [259]:
pairs_mp_df['MPTS'] = pairs_mp_df['MPTS'].astype(int)

In [12]:
from fractions import gcd
from functools import reduce
def find_gcd(list):
    x = reduce(gcd, list)
    return x

In [13]:
def fix_event_mp(event_name, mp_df, scale):
    event_mp_df = mp_df[mp_df['Event'] == event_name]
    event_mp_df
    # Identify first what is the original scale
    min_mpts = min(event_mp_df['MPTS'])
    gcd_mpts = find_gcd(event_mp_df['MPTS'])
    
    if gcd_mpts == min_mpts:
        # No ties
        original_factor = min_mpts/6
    elif 2*gcd_mpts == min_mpts:
        # Ties present, not for last awarded place
        original_factor = min_mpts/6
    elif 3*gcd_mpts == min_mpts:
        original_factor = min_mpts/9
    else:
        original_factor = min_mpts/6
        print('Check Event: ' + event_name)
        
    if scale == 'Club':
        new_factor = 1
    elif scale == 'District':
        new_factor = 1.5
    elif scale == 'County':
        new_factor = 2
    elif scale == 'Regional':
        new_factor = 3
    elif scale == 'National':
        new_factor = 4
        
    ratio = new_factor/original_factor
    print(event_name + ' ratio: ' + str(ratio))
    
    if ratio != 1:
        event_mp_df['MPTS'] = event_mp_df['MPTS']*ratio
        if scale == 'National':
            event_mp_df['Green'] = event_mp_df['MPTS']
    
        out_df = mp_df[mp_df['Event'] != event_name]
        out_df = out_df.append(event_mp_df)
        return out_df
    else:
        return mp_df

In [261]:
for i in range(len(special_event_meta_df)):
    event_name = special_event_meta_df.loc[i]['Event']
    scale = special_event_meta_df.loc[i]['Scale']
    pairs_mp_df = fix_event_mp(event_name, pairs_mp_df, scale)

Easter Congress IMP Pairsratio: 4.0
Check Event: EC Mixed
EC Mixedratio: 2.0
Open pairsratio: 2.0
Pesta Sukan Open Pairsratio: 4.0
Christmas Pairsratio: 1.5
Hongbao Pairsratio: 0.8999999999999999
NTU Open Pairsratio: 1.0
Tuesday Pairs New Year Dayratio: 1.5
Check Event: NUS Open Pairs
NUS Open Pairsratio: 1.0
Singapore Mixed Pairsratio: 4.0
Check Event: Youth Pairs
Youth Pairsratio: 2.6666666666666665
Wednesday Pairsratio: 1.0
APBF Open Pairsratio: 1.3333333333333333
APBF Open Pairs Finalsratio: 0.5454545454545455
Saturday Pairsratio: 4.0
Singapore Open Pairs Final Aratio: 4.0
Singapore Open Pairs Final Bratio: 2.6666666666666665
Wednesday Pairs (Labour Day)ratio: 1.0
Monday Pairs Vesak Dayratio: 1.5
National IMP Pairsratio: 4.0
Double 8 Pairsratio: 1.5


In [270]:
event_name = 'Youth Pairs'

In [271]:
pairs_mp_df[pairs_mp_df['Event']==event_name]

,Adjust,Cat,Date,Event,Event_ID,Factor,Factored,Green,MPTS,Name,Pair,Percent,Player ID,Position,Score,Tops
1038,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,157.333,157.333333,Jincheng Luo,13,55.61,9311.0,1,347,624
1039,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,157.333,157.333333,Linus Lee,15,55.61,9227.0,1,347,624
1040,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,120,120.000000,Keyi Pei,9,55.17,NaN,3,331,600
1041,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,96,96.000000,Yuchen Liu,7,55.00,9183.0,4,330,600
1042,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,61.3333,61.333333,Timothy Wu,2,53.83,9186.0,5,323,600
1043,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,61.3333,61.333333,Gan Yang,12,53.83,3558.0,5,323,600
1044,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,24,24.000000,Tze Lynn Kho,3,53.50,9340.0,7,321,600
1045,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,157.333,157.333333,Jun Wei Khor,13,55.61,9312.0,1,347,624
1046,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,157.333,157.333333,Gao Heng,15,55.61,9231.0,1,347,624
1047,0,NaN,01/06/2019,Youth Pairs,27,NaN,NaN,120,120.000000,Farrell Lee,9,55.17,NaN,3,331,600


In [252]:
pairs_mp_df.loc[pairs_mp_df['Event']==event_name,'MPTS'] = pairs_mp_df.loc[pairs_mp_df['Event']==event_name,'MPTS']/1.5

In [276]:
pairs_mp_df.to_csv('pairs_2019_mp.csv',index=False)

# Swiss Pairs

# Teams

In [24]:
teams_csv_list = glob.glob(csv_dir+"TS*.csv")

In [28]:
def fix_teams_header(csv_file, n_teams):
    header = pd.read_csv(csv_file, sep=';',comment='#',header=1, nrows=0, error_bad_lines=False,warn_bad_lines=False)
    data = pd.read_csv(csv_file, sep=';',comment='#',header=2, nrows=int(n_teams)-2, error_bad_lines=False,warn_bad_lines=False)
    
    diff_cols = len(data.columns) - len(header.columns)
    header_list = list(header.columns)
    idx = header_list.index('Matches')
    for i in range(0,diff_cols):
        count = str(i+2)
        idx +=1
        header_list.insert(idx,'Matches'+count)
    
    return header_list

In [23]:
def ext_winning_names_teams(df, boards, tables, scale='Club', single_winner=True):
    df_winners = df
    
    if df_winners is not None:
        df_winners1 = df_winners.drop(['Name2','Name3','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners2 = df_winners.drop(['Name1','Name3','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners3 = df_winners.drop(['Name1','Name2','Name4','Name5','Name6','Name7','Name8'], axis=1)
        df_winners4 = df_winners.drop(['Name1','Name2','Name3','Name5','Name6','Name7','Name8'], axis=1)
        df_winners5 = df_winners.drop(['Name1','Name2','Name3','Name4','Name6','Name7','Name8'], axis=1)
        df_winners6 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name7','Name8'], axis=1)
        df_winners7 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name6','Name8'], axis=1)
        df_winners8 = df_winners.drop(['Name1','Name2','Name3','Name4','Name5','Name6','Name7'], axis=1)
    
        df_winners1.rename(columns={'Name1': 'Name'},inplace=True)
        df_winners2.rename(columns={'Name2': 'Name'},inplace=True)
        df_winners3.rename(columns={'Name3': 'Name'},inplace=True)
        df_winners4.rename(columns={'Name4': 'Name'},inplace=True)
        df_winners5.rename(columns={'Name5': 'Name'},inplace=True)
        df_winners6.rename(columns={'Name6': 'Name'},inplace=True)
        df_winners7.rename(columns={'Name7': 'Name'},inplace=True)
        df_winners8.rename(columns={'Name8': 'Name'},inplace=True)
    
        out_df = pd.concat([df_winners1, df_winners2, df_winners3, df_winners4, 
                            df_winners5, df_winners6, df_winners7, df_winners8])
        return out_df
    else:
        return None

In [18]:
def ext_teams_meta(df_meta):
    event_meta_df = pd.DataFrame()
    event_meta_df['Date'] = df_meta.loc['#Date']
    event_meta_df['Event'] = df_meta.loc['#Title']
    event_meta_df['Event_ID'] = df_meta.loc['Event Number']
    event_meta_df['Director'] = df_meta.loc['#Director']
    event_meta_df['Boards'] = int(df_meta.loc['#Boards'].value)
    event_meta_df['Tables'] = int(df_meta.loc['#Tables'].value)
    event_meta_df['Rounds'] = int(df_meta.loc['#Rounds'].value)
    event_meta_df.reset_index()
    
    return event_meta_df

In [32]:
def read_teams_from_csv(csv_file):
    # Read Metadata first - take first 20 rows
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20)
    
    n_tables, n_boards, n_phantom = calc_tables_phantom(df_meta)
    
    if int(df_meta.loc['#Winners'].value)!=1:
        single_winner=False
    else:
        single_winner=True
    
    n_teams = n_tables 
    
    # Read actual results
    header_list = fix_teams_header(csv_file, n_teams)
    df = pd.read_csv(csv_file, sep=';',comment='#',header=1, names=header_list, nrows=int(n_teams), error_bad_lines=False,warn_bad_lines=False)
    if 'Position' in df['Position'].values:
        df.drop(df[df['Position']=='Position'].index, inplace=True) # Clear up extra header for 2-winner
    teams_df = df.iloc[0:int(n_teams)].sort_values('Position',ascending=True)
    
    if 'NS' in teams_df['Position'].values:
        teams_df.drop(teams_df[teams_df['Position']=='NS'].index, inplace=True) # Clear up extra header for missing phantom
        n_tables = n_tables - 1
        
    teams_df['Position'] = teams_df['Position'].astype('int') # Convert position to int
    
    out_df = ext_winning_names_teams(teams_df, n_boards, n_tables, scale='Club', single_winner=True)
    
    if out_df is not None:
        out_df['Name'] = out_df['Name'].fillna('')
        
        out_df['Player ID'] = out_df['Name'].apply(find_player_numbers, args = (players_df,))
    
        out_df['Date'] = df_meta.loc['#Date'].value
        out_df['Event'] = df_meta.loc['#Title'].value
        out_df['Event_ID'] = df_meta.loc['Event Number'].value
    
        event_meta_df = ext_teams_meta(df_meta)
    
        return out_df, event_meta_df
    else:
        event_meta_df = ext_meta(df_meta)
        return None, event_meta_df

In [21]:
teams_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"TS*.csv"):
    print(csv_file)
    df_meta = pd.read_csv(csv_file, sep=';',names=['value'],nrows=20)
    event_meta_df = ext_teams_meta(df_meta)
    
    teams_event_meta_df = teams_event_meta_df.append(event_meta_df,ignore_index=True)

D:\scba-techcomm\masterpoints\csv_2019\TSApr06-2019-146.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr07-2019-149.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr21-2019-152.csv
D:\scba-techcomm\masterpoints\csv_2019\TSAug04-2019-174.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-185.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-187.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-188.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec07-2019-189.csv
D:\scba-techcomm\masterpoints\csv_2019\TSDec15-2019-195.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan05-2019-120.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan06-2019-123.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan13-2019-128.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan19-2019-132.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan20-2019-134.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan26-2019-137.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan27-2019-140.csv
D:\scba-techcomm\masterpoints\csv_2019\T

In [33]:
out_df, meta_df = read_teams_from_csv(teams_csv_list[8])

In [51]:
out_df['Name'].replace('',np.nan, inplace=True)

In [52]:
out_df.dropna(how='all',subset=['Name','Player ID'],axis=0)

,Position,Team,Teamname,Name,Cat,Adjust,Imps,Vps,Matches,Matches2,Matches3,MPTS,Green,Player ID,Date,Event,Event_ID
1,2,1,Coup de Grace,Kelvin Ng,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,5017.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Timothy Wu,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9186.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Linus Lee,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9227.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Bryan Yang,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,9228.0,16/11/2019,H S Ho Qualifying,186
1,2,1,Coup de Grace,Xue Heng Teo,NaN,0,-24,75.38,NaN,30.59,44.79,168,0.0,5532.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Zhen Huai Toh,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9155.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Jie Min Kho,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9075.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Gideon Tan,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9209.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Jazlene Ong,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9261.0,16/11/2019,H S Ho Qualifying,186
0,1,2,Jack and Jill,Edmund Chay,NaN,0,148,108.13,49.41,NaN,58.72,264,0.0,9072.0,16/11/2019,H S Ho Qualifying,186


In [101]:
teams_mp_df = pd.DataFrame()
teams_event_meta_df = pd.DataFrame()

for csv_file in glob.glob(csv_dir+"TS*.csv"):
    print(csv_file)
    out_df, event_meta_df = read_teams_from_csv(csv_file)
    out_df.sort_values(by='Team',axis=0,inplace=True,kind='mergesort')
    out_df['Name'].replace('',np.nan, inplace=True)
    out_df.dropna(subset=['Name'],inplace=True)
    teams_mp_df = teams_mp_df.append(out_df,ignore_index=True)
    teams_event_meta_df = teams_event_meta_df.append(event_meta_df,ignore_index=True)

D:\scba-techcomm\masterpoints\csv_2019\TSApr06-2019-146.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr07-2019-149.csv
D:\scba-techcomm\masterpoints\csv_2019\TSApr21-2019-152.csv
D:\scba-techcomm\masterpoints\csv_2019\TSAug04-2019-174.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan05-2019-120.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan06-2019-123.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan13-2019-128.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan19-2019-132.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan20-2019-134.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan26-2019-137.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJan27-2019-140.csv
D:\scba-techcomm\masterpoints\csv_2019\TSJun05-2019-170.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay04-2019-158.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay11-2019-16.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay11-2019-161.csv
D:\scba-techcomm\masterpoints\csv_2019\TSMay12-2019-164.csv
D:\scba-techcomm\masterpoints\csv_2019\TS

In [103]:
teams_mp_df['Event'].value_counts()

Singapore Open Teams 2019 Qual    64
Singapore Open Team Consol RR2    43
National School Teams 2nd Day     39
Singapore Open Team Consol RR1    36
Mixed Teams 2019 RR2              31
Mixed Teams 2019 RR1              30
ABPF Open Team Trials Day 2       28
Mixed Team Trials Day 2           26
APBF Open Team Trials             26
Mixed Trials 2019                 24
H S Ho Qualifying                 20
Singapore Open Team Final 2019    13
ABPF Open Team Trials Day 4       11
Pesta Sukan Open Team Final       11
ABPF Open Team Trials              9
Mixed Team Trials Final 2019       8
EC Open Teams Finals               3
Name: Event, dtype: int64

In [102]:
teams_mp_df.to_csv('teams_mp_2019.csv',index=False)

In [84]:
filter_col = [col for col in teams_mp_df if col.startswith('Matches')]
filter_col_a = filter_col + ['Team']

In [85]:
filter_col

['Matches',
 'Matches10',
 'Matches2',
 'Matches3',
 'Matches4',
 'Matches5',
 'Matches6',
 'Matches7',
 'Matches8',
 'Matches9']

In [70]:
teams_mp_df[teams_mp_df['Event']=='Handicap Teams'][filter_col].groupby('Team').sum()

,Vps
Team,
1,443.40
2,287.92
3,182.94
4,21.78
5,132.04
6,79.93
7,433.54


In [86]:
tmp = teams_mp_df[teams_mp_df['Event']=='Handicap Teams'][filter_col_a]

In [77]:
def return_win(x):
    if x > 10:
        return 1
    else:
        return 0

In [90]:
tmp.loc[:, tmp.columns != 'Team'] = tmp[filter_col].applymap(return_win)

In [100]:
tmp[tmp['Team']==6]

,Matches,Matches10,Matches2,Matches3,Matches4,Matches5,Matches6,Matches7,Matches8,Matches9,Team
88,0,0,0,0,0,1,0,0,0,0,6
89,0,0,0,0,0,1,0,0,0,0,6
109,0,0,0,0,0,0,0,0,0,0,6
110,0,0,0,0,0,0,0,0,0,0,6
131,0,0,0,0,1,0,0,0,0,0,6
132,0,0,0,0,1,0,0,0,0,0,6
133,0,0,0,0,1,0,0,0,0,0,6
156,0,0,0,0,0,0,0,0,0,0,6
